In [41]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import lightgbm as lgb

In [42]:
# Display Options of DataFrame
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)

In [43]:
# Loading Swap Rate Dataset
data = pd.read_csv("../cleanedData/merged.csv")

In [44]:
data['CMS Tenor'] = data['CMS Tenor'].str.removesuffix('y')
data['CMS Tenor'] = data['CMS Tenor'].astype(int)

In [45]:
# Find lag 1 by shifting the amount of data in one date
delay = len(data.loc[data['Value Date'] == "2022-09-02"])

# Realized Swap Rate Prev
data['Swap Rate Lag 1 (Real)'] = data['Swap Rate'].shift(delay)

In [46]:
data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,-100,-0.0100,-227907.098775,1.0,2023-09-04,10.0,1.962246,2,0.029155,0.019155,0.0042,0.0379,0.0337,6,5,0.014955,0.018745,False,0.014955,0.177525,NaN
1,2022-09-02,dummyTrade1,USD,-50,-0.0050,-222208.400967,1.0,2023-09-04,10.0,-3.812341,2,0.029155,0.024155,0.0042,0.0379,0.0337,6,5,0.019955,0.013745,False,0.013745,0.177525,NaN
2,2022-09-02,dummyTrade1,USD,-25,-0.0025,-218960.927995,1.0,2023-09-04,10.0,4.471006,2,0.029155,0.026655,0.0042,0.0379,0.0337,6,5,0.022455,0.011245,False,0.011245,0.177525,NaN
3,2022-09-02,dummyTrade1,USD,-10,-0.0010,-216872.430106,1.0,2023-09-04,10.0,4.333398,2,0.029155,0.028155,0.0042,0.0379,0.0337,6,5,0.023955,0.009745,False,0.009745,0.177525,NaN
4,2022-09-02,dummyTrade1,USD,-5,-0.0005,-216146.310328,1.0,2023-09-04,10.0,5.679687,2,0.029155,0.028655,0.0042,0.0379,0.0337,6,5,0.024455,0.009245,False,0.009245,0.177525,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.0010,-547852.055930,15.0,2039-01-12,8.0,5.007952,5,0.030386,0.031386,0.0044,0.0397,0.0353,6,10,0.026986,0.008314,False,0.008314,0.253906,0.030687
3919100,2024-01-12,dummyTrade9,USD,25,0.0025,-521228.587618,15.0,2039-01-12,8.0,5.131505,5,0.030386,0.032886,0.0044,0.0397,0.0353,6,10,0.028486,0.006814,False,0.006814,0.253906,0.030687
3919101,2024-01-12,dummyTrade9,USD,50,0.0050,-464905.982097,15.0,2039-01-12,8.0,3.348622,5,0.030386,0.035386,0.0044,0.0397,0.0353,6,10,0.030986,0.004314,False,0.004314,0.253906,0.030687
3919102,2024-01-12,dummyTrade9,USD,75,0.0075,-391315.041929,15.0,2039-01-12,8.0,1.063634,5,0.030386,0.037886,0.0044,0.0397,0.0353,6,10,0.033486,0.001814,False,0.001814,0.253906,0.030687


In [47]:
zero_subset = data['Zero Rate Shock'] == 0
shock_subset = -zero_subset
zero = data[zero_subset].reset_index(drop = True)
shock = data[shock_subset].reset_index(drop = True)

In [48]:
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518


In [49]:
delay = len(zero.loc[zero['Value Date'] == "2022-09-02"])
zero['Vols Prev'] = zero['Vols'].shift(delay)
zero['Vols Change'] = zero['Vols'] - zero['Vols Prev']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271


In [51]:
# Find Risk of RAN notes contributed by swaption IV changes

# change in TV caused by vega (t=0) = previous vega (t-1)* vol change
zero['Vega Lag 1']  = zero['Vega'].shift(delay)
zero['Vega Risk Exposure'] = zero['Vols Change'] * zero['Vega Lag 1']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237


In [52]:
zero['Vega Risk Exposure'].describe()

count    325620.000000
mean          0.026966
std          22.079677
min       -3300.852081
25%          -0.007612
50%           0.000000
75%           0.008292
max        5232.585440
Name: Vega Risk Exposure, dtype: float64

In [53]:
zero['Swap Rate Change'] = zero['Swap Rate'] - zero['Swap Rate Lag 1 (Real)']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412,-0.000164
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546,-0.000164
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309,-0.000691
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237,-0.000691


In [54]:
zero['TV Lag 1'] = zero['TV'].shift(delay)
zero['TV Change'] = zero['TV'] - zero['TV Lag 1'] 
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
0,2022-09-02,dummyTrade1,USD,0,0.0,-215402.656152,1.0,2023-09-04,10.0,5.016274,2,0.029155,0.029155,0.0042,0.0379,0.0337,6,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.0,-616299.369241,1.0,2023-09-04,10.0,-59.820414,5,0.027246,0.027246,0.0044,0.0397,0.0353,4,10,0.022846,0.012454,False,0.012454,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.0,-608678.010978,1.0,2023-09-04,10.0,7.762595,10,0.027377,0.027377,0.0046,0.0417,0.0371,4,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.0,-609360.662412,1.0,2023-09-04,10.0,25.329779,10,0.027377,0.027377,0.0046,0.0417,0.0371,3,10,0.022777,0.014323,False,0.014323,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.0,-215836.787208,1.0,2023-09-04,10.0,5.409384,2,0.029155,0.029155,0.0042,0.0379,0.0337,4,5,0.024955,0.008745,False,0.008745,0.177525,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.0,-200752.707821,15.0,2039-01-12,8.0,-0.277974,10,0.031712,0.031712,0.0046,0.0417,0.0371,4,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.156327,-0.000412,-0.000164,-201031.681080,278.973259
326588,2024-01-12,dummyTrade6,USD,0,0.0,-200966.214770,15.0,2039-01-12,8.0,-0.296353,10,0.031712,0.031712,0.0046,0.0417,0.0371,3,5,0.027112,0.009988,False,0.009988,0.253906,0.031876,0.251271,0.002636,-0.207006,-0.000546,-0.000164,-201241.288696,275.073926
326589,2024-01-12,dummyTrade7,USD,0,0.0,-507396.663276,15.0,2039-01-12,8.0,5.056103,2,0.029827,0.029827,0.0042,0.0379,0.0337,6,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,5.049607,0.013309,-0.000691,-509747.568328,2350.905053
326590,2024-01-12,dummyTrade8,USD,0,0.0,-508418.039331,15.0,2039-01-12,8.0,4.911479,2,0.029827,0.029827,0.0042,0.0379,0.0337,4,10,0.025627,0.008073,False,0.008073,0.253906,0.030518,0.251271,0.002636,4.263389,0.011237,-0.000691,-510774.029616,2355.990285


In [55]:
#zero['TV Change'].describe()

count    325620.000000
mean        136.406926
std        9325.497424
min     -110564.166917
25%       -2034.187152
50%           1.021570
75%        2354.574763
max      181650.015378
Name: TV Change, dtype: float64

In [63]:
#zero[zero['TV Change'] == zero['TV Change'].max()]
#zero[(zero['Value Date'] == '2023-04-24') & (zero['Trade Name'] == "dummyTrade12")]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
231338,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,10.0,-5.481733,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,0.997175,0.033431,1.081294,-0.084119,-3.991010,0.335720,-0.000054,-433369.130798,-110564.166917
231350,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,15.0,-3.491122,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,0.815910,0.033431,1.027118,-0.211208,90.739206,-19.164864,-0.000054,-433369.130798,-110564.166917
231362,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,1.5,0.000000,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,2.340865,0.033431,2.389034,-0.048169,0.000000,-0.000000,-0.000054,-433369.130798,-110564.166917
231374,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,1.0,0.002057,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,2.601974,0.033431,2.646886,-0.044912,-0.040527,0.001820,-0.000054,-433369.130798,-110564.166917
231386,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,2.0,-0.511269,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,2.165516,0.033431,2.201660,-0.036143,-0.999534,0.036127,-0.000054,-433369.130798,-110564.166917
231398,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,3.0,0.450969,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,1.876024,0.033431,1.902506,-0.026483,-3.046169,0.080670,-0.000054,-433369.130798,-110564.166917
231410,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,4.0,1.772880,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,1.657745,0.033431,1.679389,-0.021644,-3.822832,0.082741,-0.000054,-433369.130798,-110564.166917
231422,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,5.0,3.785322,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,1.470820,0.033431,1.510690,-0.039870,-4.797125,0.191261,-0.000054,-433369.130798,-110564.166917
231434,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.0,2024-08-30,8.0,-3.189103,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,1.127362,0.033431,1.204017,-0.076655,-5.480989,0.420145,-0.000054,-433369.130798,-110564.166917
231446,2023-08-30,dummyTrade11,USD,0,0.0,-543933.297715,1.5,2025-02-28,10.0,98.839208,10,0.033378,0.033378,0.0046,0.0417,0.0371,4,10,0.028778,0.008322,False,0.008322,0.890350,0.033431,0.972239,-0.081888,43.667345,-3.575853,-0.000054,-433369.130798,-110564.166917


In [ ]:
# Completed Zero Rate preprocessing, Next Up is Shock  